In [149]:
import pandas as pd
from rdflib import Graph
from pathlib import Path


# EDA & Nettoyage

### Chargement des CSV (Philadelphia)

In [189]:
# Ridership par mode
df_mode = pd.read_csv(
    "../data/Average_Daily_Ridership_By_Mode - City of Philadelphia.csv"
)



df_mode = df_mode.rename(columns={
    "Mode": "mode",
    "Average_Daily_Ridership": "ridership",
    "Calendar_Year": "year",
    "Calendar_Month": "month"
})


df_mode = df_mode[
    [ "year", "month", "mode", "ridership"]

]
df_mode["city"] = 'Philadelphia'
df_mode.info()
df_mode.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492 entries, 0 to 491
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   year       492 non-null    int64 
 1   month      492 non-null    int64 
 2   mode       492 non-null    object
 3   ridership  492 non-null    int64 
 4   city       492 non-null    object
dtypes: int64(3), object(2)
memory usage: 19.3+ KB


,year,month,mode,ridership,city
0,2019,1,Bus,459160,Philadelphia
1,2019,1,CCT,4294,Philadelphia
2,2019,1,Heavy Rail,296709,Philadelphia
3,2019,1,Regional Rail,122856,Philadelphia
4,2019,1,Trackless Trolley,16853,Philadelphia


In [151]:
# Check for missing values ridership par mode
df_mode.isna().sum()

year         0
month        0
mode         0
ridership    0
dtype: int64

In [190]:
# Ridership quotidien par route
df_route = pd.read_csv(
    "../data/Average_Daily_Ridership_By_Route - City of Philadelphia.csv"
)


df_route = df_route.rename(columns={
    "Route": "route_id",
    "Average_Daily_Ridership": "ridership",
    "Calendar_Year": "year",
    "Calendar_Month": "month"
})


df_route = df_route[
    [ "year", "month", "route_id", "ridership"]
]
df_route["city"] = 'Philadelphia'
df_route.info()
df_route.head(30)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10994 entries, 0 to 10993
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   year       10994 non-null  int64 
 1   month      10994 non-null  int64 
 2   route_id   10994 non-null  object
 3   ridership  10994 non-null  int64 
 4   city       10994 non-null  object
dtypes: int64(3), object(2)
memory usage: 429.6+ KB


,year,month,route_id,ridership,city
0,2019,8,59,3811,Philadelphia
1,2019,8,60,8550,Philadelphia
2,2019,8,61,3563,Philadelphia
3,2019,8,62,226,Philadelphia
4,2019,8,64,4335,Philadelphia
5,2019,8,65,7052,Philadelphia
6,2019,8,66,8839,Philadelphia
7,2019,8,67,2674,Philadelphia
8,2019,8,68,2004,Philadelphia
9,2019,8,70,5029,Philadelphia


In [153]:
# Check for missing values Ridership par route
df_route.isna().sum()

year         0
month        0
route_id     0
ridership    0
dtype: int64

In [191]:
df_mode.to_csv("../data/philadelphia/Fact_mode_phila.csv", index=False)
df_route.to_csv("../data/philadelphia/Fact_route_phila.csv", index=False)

### Chargement des dateset (Chicago)

#### Excel file 

In [155]:
df_chicago_mode = pd.read_excel('../data/cta-ridership-daily-boarding-totals-20260203-69820a3f9df63091665572 (2).xlsx')

In [156]:
df_chicago_mode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9100 entries, 0 to 9099
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   service_date    9100 non-null   datetime64[ns]
 1   day_type        9100 non-null   object        
 2   bus             9100 non-null   int64         
 3   rail_boardings  9100 non-null   int64         
 4   total_rides     9100 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 355.6+ KB


In [157]:
df_chicago_mode.value_counts('service_date')

service_date
2001-01-01    1
2001-01-02    1
2001-01-03    1
2001-01-04    1
2001-01-05    1
             ..
2025-11-26    1
2025-11-27    1
2025-11-28    1
2025-11-29    1
2025-11-30    1
Name: count, Length: 9100, dtype: int64

In [192]:
df_chicago_mode["service_date"] = pd.to_datetime(df_chicago_mode["service_date"])
df_chicago_mode["year"] = df_chicago_mode["service_date"].dt.year
df_chicago_mode["month"] = df_chicago_mode["service_date"].dt.month

# Filtrer 2019 → 2025
df_chicago_mode = df_chicago_mode[
    (df_chicago_mode['service_date'] >= '2019-01-01') &
    (df_chicago_mode['service_date'] <= '2025-12-31')
]

# Agréger par mois
df_chicago_mode_monthly = (
    df_chicago_mode
    .groupby(["year", "month"], as_index=False)
    .agg({
        "bus": "mean",
        "rail_boardings": "mean"
    })
)
df_chicago_mode_monthly["bus"] = df_chicago_mode_monthly["bus"].round(0).astype(int)
df_chicago_mode_monthly["rail_boardings"] = df_chicago_mode_monthly["rail_boardings"].round(0).astype(int)
df_chicago_mode_monthly["city"] = 'Chicago'



In [193]:
df_chicago_mode.value_counts('service_date')

service_date
2019-01-01    1
2019-01-02    1
2019-01-03    1
2019-01-04    1
2019-01-05    1
             ..
2025-11-26    1
2025-11-27    1
2025-11-28    1
2025-11-29    1
2025-11-30    1
Name: count, Length: 2526, dtype: int64

In [194]:
# df_chicago_mode.head()
df_chicago_mode_monthly.head()


,year,month,bus,rail_boardings,city
0,2019,1,563028,518350,Chicago
1,2019,2,652810,584176,Chicago
2,2019,3,653779,581264,Chicago
3,2019,4,664563,612855,Chicago
4,2019,5,682093,619580,Chicago


In [161]:
df_chicago_mode_monthly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   year            83 non-null     int32
 1   month           83 non-null     int32
 2   bus             83 non-null     int64
 3   rail_boardings  83 non-null     int64
dtypes: int32(2), int64(2)
memory usage: 2.1 KB


In [195]:
df_chicago_mode_monthly.to_csv("../data/chicago/Fact_mode_chicago.csv", index=False)

#### Convertir RDF to CSV

In [ ]:
import pandas as pd
import glob
from lxml import etree

# Chemin vers tes fichiers RDF
RDF_PATH = "../data/chicago_route_RDF/*.rdf"

# Namespaces RDF
NS = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "ds": "https://data.cityofchicago.org/resource/jyb9-n7fm/"
}

def parse_rdf(file_path):
    tree = etree.parse(file_path)
    root = tree.getroot()

    records = []

    for desc in root.findall("rdf:Description", namespaces=NS):
        record = {
            "route": desc.findtext("ds:route", namespaces=NS),
            "date": desc.findtext("ds:date", namespaces=NS),
            "daytype": desc.findtext("ds:daytype", namespaces=NS),
            "rides": desc.findtext("ds:rides", namespaces=NS),
        }
        records.append(record)

    return pd.DataFrame(records)

# Lire tous les fichiers RDF
dfs = []
for file in glob.glob(RDF_PATH):
    print(f"Lecture : {file}")
    dfs.append(parse_rdf(file))

# Fusion
df_chicago_route = pd.concat(dfs, ignore_index=True)

# Nettoyage
df_chicago_route["date"] = pd.to_datetime(df_chicago_route["date"])
df_chicago_route["rides"] = df_chicago_route["rides"].astype(int)
df_chicago_route["route"] = df_chicago_route["route"].astype(str)




Lecture : ../data/chicago_route_RDF\rdf_CTA__Ridership__Daily_by_Route_routes_1.rdf
Lecture : ../data/chicago_route_RDF\rdf_CTA__Ridership__Daily_by_Route_routes_10.rdf
Lecture : ../data/chicago_route_RDF\rdf_CTA__Ridership__Daily_by_Route_routes_11.rdf
Lecture : ../data/chicago_route_RDF\rdf_CTA__Ridership__Daily_by_Route_routes_12.rdf
Lecture : ../data/chicago_route_RDF\rdf_CTA__Ridership__Daily_by_Route_routes_13.rdf
Lecture : ../data/chicago_route_RDF\rdf_CTA__Ridership__Daily_by_Route_routes_14.rdf
Lecture : ../data/chicago_route_RDF\rdf_CTA__Ridership__Daily_by_Route_routes_15.rdf
Lecture : ../data/chicago_route_RDF\rdf_CTA__Ridership__Daily_by_Route_routes_16.rdf
Lecture : ../data/chicago_route_RDF\rdf_CTA__Ridership__Daily_by_Route_routes_17.rdf
Lecture : ../data/chicago_route_RDF\rdf_CTA__Ridership__Daily_by_Route_routes_18.rdf
Lecture : ../data/chicago_route_RDF\rdf_CTA__Ridership__Daily_by_Route_routes_19.rdf
Lecture : ../data/chicago_route_RDF\rdf_CTA__Ridership__Daily_by_R

In [164]:
df_chicago_route.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092474 entries, 0 to 1092473
Data columns (total 4 columns):
 #   Column   Non-Null Count    Dtype         
---  ------   --------------    -----         
 0   route    1092474 non-null  object        
 1   date     1092474 non-null  datetime64[ns]
 2   daytype  1092474 non-null  object        
 3   rides    1092474 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 33.3+ MB


In [165]:
df_chicago_route.isna().sum()

route      0
date       0
daytype    0
rides      0
dtype: int64

In [166]:
df_chicago_route.head()

,route,date,daytype,rides
0,1,2004-06-16,W,3174
1,100,2021-06-29,W,176
2,100,2018-05-21,W,497
3,106,2009-02-25,W,3146
4,100,2021-12-30,W,204


In [198]:
# Convertir la date
df_chicago_route["date"] = pd.to_datetime(df_chicago_route["date"])

# Créer année & mois
df_chicago_route["year"] = df_chicago_route["date"].dt.year
df_chicago_route["month"] = df_chicago_route["date"].dt.month

# Filtrer 2019 → 2025
df_chicago_route = df_chicago_route[
    (df_chicago_route["date"].dt.year >= 2019) &
    (df_chicago_route["date"].dt.year <= 2025)
]

# Agrégation mensuelle
df_chicago_route_monthly = (
    df_chicago_route
    .groupby(["year", "month", "route"], as_index=False)
    .agg({"rides": "mean"})
    .rename(columns={"rides": "ridership"})
)

df_chicago_route_monthly["ridership"] = df_chicago_route_monthly["ridership"].round(0).astype(int)
df_chicago_route_monthly["city"] = 'Chicago'


In [199]:
df_chicago_route_monthly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10465 entries, 0 to 10464
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   year       10465 non-null  int32 
 1   month      10465 non-null  int32 
 2   route      10465 non-null  object
 3   ridership  10465 non-null  int64 
 4   city       10465 non-null  object
dtypes: int32(2), int64(1), object(2)
memory usage: 327.2+ KB


In [200]:
# Export CSV
output_path = "../data/chicago/Fact_route_chicago.csv"
df_chicago_route_monthly.to_csv(output_path, index=False)

### Modélisation des tables de dimensions

In [170]:
dim_year = pd.DataFrame({
    "year": list(range(2019, 2026))
})


In [171]:
dim_year.head(20)

,year
0,2019
1,2020
2,2021
3,2022
4,2023
5,2024
6,2025


In [172]:
dim_year.to_csv("../data/dim_table/Dim_year.csv", index=False)

In [173]:
dim_month = pd.DataFrame({
    "month": list(range(1, 13)),
    "month_name": [
        "Jan", "Feb", "Mar", "Apr", "May", "Jun",
        "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"
    ],
    "month_order": list(range(1, 13))
})


In [174]:
dim_month.head(20)

,month,month_name,month_order
0,1,Jan,1
1,2,Feb,2
2,3,Mar,3
3,4,Apr,4
4,5,May,5
5,6,Jun,6
6,7,Jul,7
7,8,Aug,8
8,9,Sep,9
9,10,Oct,10


In [175]:
dim_month.to_csv("../data/dim_table/Dim_month.csv", index=False)